In [1]:
import nltk
import numpy as np
import random
import string

# 1 DEFINICIÓN CORPUS

In [2]:
f=open(r'corpus.txt','r',errors = 'ignore')
raw=f.read()
raw

'Belanova es una banda de mÃºsica electrÃ³nica originaria de MÃ©xico, que ha lanzado varios Ã¡lbumes a lo largo de su carrera. AquÃ\xad se describe cada uno de sus Ã¡lbumes, incluyendo algunas de sus canciones mÃ¡s populares:\n\n"Cocktail" (2003): Este fue el Ã¡lbum debut de Belanova. El Ã¡lbum incluye canciones como "Me pregunto", "Tus ojos" y "Suele pasar". "Me pregunto" fue uno de los sencillos mÃ¡s populares del Ã¡lbum, y el video musical ganÃ³ varios premios.\n\n"Dulce Beat" (2005): Este es el Ã¡lbum mÃ¡s exitoso de Belanova hasta la fecha. El Ã¡lbum incluye canciones como "Por ti", "Rosa pastel", "Baila mi corazÃ³n" y "Toma mi mano". "Por ti" y "Rosa pastel" se convirtieron en Ã©xitos en MÃ©xico y en otros paÃ\xadses de habla hispana.\n\n"FantasÃ\xada Pop" (2007): Este Ã¡lbum muestra una evoluciÃ³n en el sonido de Belanova, con un estilo mÃ¡s electrÃ³nico y bailable. El Ã¡lbum incluye canciones como "Baila conmigo", "Cada que..." y "Baila mi corazÃ³n" (remix). "Baila conmigo" fue

# 2a Preprocesamiento del Texto con NTLK CORPUS

In [3]:
raw=raw.lower()# convertir en minúscula
#nltk.download('punkt') # Instalar módulo punkt si no está ya instalado (solo ejecutar la primera vez)
#nltk.download('wordnet') # Instalar módulo wordnet si no está ya instalado (solo ejecutar la primera vez)
sent_tokens = nltk.sent_tokenize(raw)# Convierte el CORPUS a una lista de sentencias
word_tokens = nltk.word_tokenize(raw)# Convierte el CORPUS a una lista de palabras
lemmer = nltk.stem.WordNetLemmatizer()

#WordNet diccionario semántico incluido en NLTK
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# 2b PREPROCESAMIENTO DEL TEXTO + 3 Evaluar Similitud MENSAJE USUARIO - CORPUS

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

#Función para determinar la similitud del texto insertado y el corpus
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response) #Añade al corpus la respuesta de usuario al final
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish'))
    tfidf = TfidfVec.fit_transform(sent_tokens)
    # 3 EVALUAR SIMILITUD DE COSENO ENTRE MENSAJE USUARIO (tfidf[-1]) y el CORPUS (tfidf)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if(req_tfidf==0):
        robo_response=robo_response+"Lo siento, no te he entendido. Si no puedo responder a lo que busca póngase en contacto con soporte@soporte.com"
        return robo_response

    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

# 4 DEFINICIÓN DE COINCIDENCIAS MANUAL

In [5]:
SALUDOS_INPUTS = ("hola", "buenas", "saludos", "qué tal", "hey","buenos dias",)
SALUDOS_OUTPUTS = ["Hola", "Hola, ¿Qué tal?", "Hola, ¿Cómo te puedo ayudar?", "Hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

# 5 GENERACIÓN DE RESPUESTA

In [ ]:
flag=True
print("Chatbot: Mi nombre es ROBOT. Contestaré a tus preguntas acerca de la discografía de Belanova. Si quieres salir, escribe 'salir' ")
while(flag==True):
    print("")
    user_response = input("Usuario: ")
    user_response = user_response.lower() #Convertimos a minúscula
    
    if(user_response!='salir'):
        
        if(user_response=='gracias' or user_response=='muchas gracias'): #Se podría haber definido otra función de coincidencia manual
            flag=True
            print("Chatbot: No hay de qué")
            
        else:
            if(saludos(user_response)!=None): #Si la palabra insertada por el usuario es un saludo (Coincidencias manuales definidas previamente)
                print("ROBOT: "+saludos(user_response))
                
            else: #Si la palabra insertada no es un saludo --> CORPUS
                print("ROBOT: ",end="") 
                print(response(user_response))
                sent_tokens.remove(user_response) # para eliminar del corpus la respuesta del usuario y volver a evaluar con el CORPUS limpio
    else:
        flag=False
        print("Chatbot: Nos vemos pronto, ¡cuídate!")

Chatbot: Mi nombre es ROBOT. Contestaré a tus preguntas acerca de la discografía de Belanova. Si quieres salir, escribe 'salir' 

Usuario: quién es belanova?
ROBOT: "cerrando ciclos" (2018): este es el ã¡lbum mã¡s reciente de belanova.

Usuario: a qué se dedica?
ROBOT: Lo siento, no te he entendido. Si no puedo responder a lo que busca póngase en contacto con soporte@soporte.com

Usuario: cuentame acerca de belanova?
ROBOT: "cerrando ciclos" (2018): este es el ã¡lbum mã¡s reciente de belanova.

Usuario: cuáles son sus grandes éxitos?
ROBOT: Lo siento, no te he entendido. Si no puedo responder a lo que busca póngase en contacto con soporte@soporte.com

Usuario: qué canciones tiene?
ROBOT: el ã¡lbum incluye canciones como "nada de mã¡s", "hasta el final" y "hoy".

Usuario: hola
ROBOT: Hola, ¿Cómo te puedo ayudar?

Usuario: integrantes de belanova
ROBOT: en resumen, los integrantes de belanova siguen activos en la mãºsica y en otros proyectos creativos, tanto en conjunto como en solitario